## Import

In [1]:
# import trimesh as tm
# import topogenesis as tg
import numpy as np

# import os
import math
import ortools

from ortools.linear_solver import pywraplp

## Loading context

In [2]:
# path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
# buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height TODO: will be USER INPUT)

vox_size = 10
fh = 3.6 # floor height

# max_extents_height = abs(buildingplot.bounds[0][2]) + abs(buildingplot.bounds[1][2])
max_extents_height = 36
# max_extents_width = abs(buildingplot.bounds[0][0]) + abs(buildingplot.bounds[1][0])
max_extents_width = 140
# max_extents_length = abs(buildingplot.bounds[0][1]) + abs(buildingplot.bounds[1][1])
max_extents_length = 150

## FSI presets

In [3]:
# base = buildingplot.apply_transform(tm.transformations.projection_matrix((0,0,0), (0,0,-1))) # project mesh on z plane to get footprint <-- TODO: easier way?
# plot_area = base.area/2 # area calculates both sides of the flattened mesh, we need only 1
plot_area = 18000
FSI_req = 4 # goal for the FSI TODO: USER INPUT

## PV presets

In [4]:
# Annual solar energy output of PV is calculated as follows:
# E(kwh) = A*r*H*PR where A = total solar panel area, r = solar panel yield/efficiency, H = Annual average solar radiation, PR = performance ratio
# from this site TODO find better source: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 

# r = 0.15 # yield/efficiency estimate
# H = 1050 # to be updated, currently wikipedia estimate. kWh.y
# PR = 0.75 # default value, estimate performance ratio influenced by angle
# PV_req = 20 # kWh/m2/year TODO: change to actual expected value from included colors + find source on this value

## ORTools - variables

In [11]:
solver = pywraplp.Solver.CreateSolver("SCIP") # GLOP and SCIP get different and sometimes 'wrong' results, also different results at different times (restarting kernel seems to help)
# TODO:--> different solvers? GUROBI? CP-SAT? constraint programming not MILP?
infinity = math.inf

a = solver.IntVar(1.0, infinity, 'floors') # number of floors
b = solver.IntVar(10.0, infinity, 'length') # building length
c = solver.IntVar(10.0, infinity, 'width') # building width
# d = solver.IntVar(0.0, 90.0, 'rotation') # building orientation
e = solver.IntVar(0, infinity, 'aux') # e is an auxilliary variable for the area in order to change the problem from quadratic to linear

print(solver.NumVariables())

4


## ORTools - constraints

In [6]:
solver.Add(a * fh <= max_extents_height) # ensures height limit is not exceeded
solver.Add(e <= 100*b) 
solver.Add(e <= 100*c)
solver.Add(e >= 10*b + 10*c -100)
solver.Add(e >= 100*b + 100*c - 10000) # assuming max width and max height are 100 TODO: adapt to compensate for angle

# not sure if these work as it they are supposed to right now:
# solver.Add(b * math.cos(d.solution_value()) + c * math.sin(d.solution_value()) <= max_extents_length) # constrains the length for the given oreientation
# solver.Add(c * math.cos(d.solution_valuen()) + b * math.n(d.solution_extents_width) # constrains the width for the given orientation

print(solver.NumConstraints())

5


## objective function

In [7]:
def FSI_lowres(floors, width, length, plot = plot_area, goal = FSI_req):
    FSI = (floors * width * length) / plot
    # FSI = floors / plot
    FSI_normalized = 1/(1+(FSI - goal)**2)
    # FSI_normalized = (1+(FSI - goal))
    if FSI >= goal:
        FSI_normalized = 1 # if FSI is more than needed, normalized value is 1
    return FSI, FSI_normalized # gives a rough estimate of the FSI, and normalizes this value on 0-1 depending on how close to the goal it is. 1 if it exceeds the goal

# def FSI_linear(np.piecewise(FSI_lowres, [x < 0, x >= 0], x, x)):

def AREA(length, width):
    area = length*width
    return area

x = np.linspace(-2.5, 2.5, 6)

print(np.piecewise(x, [x < 0, x >= 0], [lambda x: -x, lambda x: 1]))
x

[2.5 1.5 0.5 1.  1.  1. ]


array([-2.5, -1.5, -0.5,  0.5,  1.5,  2.5])

## ORTools - solving

In [8]:
# solver.Maximize(1/(1+(a-FSI_req)**2))
# solver.Maximize(FSI_lowres(a.solution_value(), b.solution_value(), c.solution_value())[1]) # <-- gives a result but uses only the lower bound of the variables and stops 
# solver.Maximize(a*b) # <-- this one gives type error (it probably cant pass 'variable' type to the FSI_lowres definition)
# solver.Maximize(FSI_lowres(a,b,c)[1]) # <-- this is what I want in the end
# solver.Maximize(FSI_lowres(a)[1])
solver.Maximize(AREA(a,b))

# I might need to use MPSolverParameters.GetIntegerParam to pass current values to the FSI definition, but I don't understand the documentation on this: 
# paras = pywraplp.MPSolverParameters().GetIntegerParam() 

# another thing that may be the root cause of the issue is that by definition multiplying variables in the objective is not possible since then it is no longer a linear problem?
# this would mean I need to TODO: linearize the objective function for FSI (and PV)?

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('floors =', a.solution_value())
    print('width =', b.solution_value())
    print('length =', c.solution_value())
    # print('rotation =', d.solution_value())
else:
    print('The problem does not have an optimal solution.', status, pywraplp.Solver.OPTIMAL)

TypeError: 